In [1]:
filename = 'nyu_depth_v2_labeled.mat'

In [69]:
import h5py
import numpy as np

class Loader:
    def __init__(self, file='./nyu_depth_v2_labeled.mat', train_ratio=0.9, batch_size=8, allow_smaller_final_batch=False):
        self.__datafile = h5py.File(file)
        self.__indices = np.arange(self.__datafile['images'].shape[0])
        np.random.shuffle(self.__indices)
        cut_idx = int(train_ratio*self.__indices.shape[0])
        self.__train_indices = self.__indices[:cut_idx]
        self.__test_indices = self.__indices[cut_idx:]
        self.__bs = batch_size
        self.__train_pool = np.array(self.__train_indices)
        self.__test_pool = np.array(self.__test_indices)
        self.__smaller_final = allow_smaller_final_batch
    
    @staticmethod
    def __get_batch(indices, batch_size):
        return np.random.choice(indices, size=batch_size, replace=False)
    @staticmethod
    def __read_batch(batch, f):
        return f['images'][np.sort(batch),...].transpose((0,3,2,1)), f['depths'][np.sort(batch),...].transpose((0,2,1))
    
    def train_batch(self):
        bs = self.__bs
        if self.__train_pool.shape[0]<self.__bs:
            if self.__smaller_final:
                bs = self.__train_pool.shape[0]
            else:
                self.__train_pool = np.array(self.__train_indices)
        batch = self.__get_batch(self.__train_pool, bs)
        self.__train_pool = np.setdiff1d(self.__train_pool, batch, assume_unique=True)
        if self.__train_pool.shape[0]==0:
            self.__train_pool = np.array(self.__train_indices)
        return self.__read_batch(batch, self.__datafile)
    
    def test_batch(self):
        bs = self.__bs
        if self.__test_pool.shape[0]<self.__bs:
            if self.__smaller_final:
                bs = self.__test_pool.shape[0]
            else:
                self.__test_pool = np.array(self.__test_indices)
        batch = self.__get_batch(self.__test_pool, bs)
        self.__test_pool = np.setdiff1d(self.__test_pool, batch, assume_unique=True)
        if self.__test_pool.shape[0]==0:
            self.__test_pool = np.array(self.__test_indices)
        return self.__read_batch(batch, self.__datafile)
    
    def size(self):
        return len(self.__train_indices), len(self.__test_indices)
    
    def n_batches(self):
        if self.__smaller_final:
            op = np.ceil
        else:
            op = np.floor
        return int(op(len(self.__train_indices)/self.__bs)), int(op(len(self.__test_indices)/self.__bs))

In [75]:
loader = Dataloader(batch_size=8, allow_smaller_final_batch=False)

In [76]:
loader.n_batches()

(163, 18)

In [62]:
loader.size()

(1304, 145)

In [10]:
keras.backend.clear_session()
X = keras.layers.Input(shape=(304,228,3), name='Input')
Coarse1 = keras.layers.LeakyReLU()(
            keras.layers.MaxPool2D(pool_size=2)(
                keras.layers.Conv2D(filters=96, 
                                    kernel_size=11, 
                                    strides=4)(X)))
Coarse2 = keras.layers.LeakyReLU()(
            keras.layers.MaxPool2D(pool_size=2)(
                keras.layers.Conv2D(filters=256,
                                    kernel_size=5,
                                    padding='same')(Coarse1)))
Coarse3 = keras.layers.LeakyReLU()(
            keras.layers.Conv2D(filters=384,
                              kernel_size=3,
                              padding='same')(Coarse2))
Coarse4 = keras.layers.LeakyReLU()(
            keras.layers.Conv2D(filters=384,
                              kernel_size=3,
                              padding='same')(Coarse3))
Coarse5 = keras.layers.LeakyReLU()(
            keras.layers.MaxPool2D(pool_size=2)(
                keras.layers.Conv2D(filters=256,
                                  kernel_size=3,
                                  padding='same')(Coarse4)))
Coarse6 = keras.layers.Dropout(0.1)(
            keras.layers.LeakyReLU()(
                keras.layers.Dense(units=4096)(Coarse5)))
#Coarse7 = 


In [11]:
Coarse6.shape

TensorShape([Dimension(None), Dimension(9), Dimension(6), Dimension(4096)])

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (640,480))
Coarse1 = tf.layers.max_pooling2d(
    inputs = tf.layers.conv2d(
                inputs=X, 
                filters=96, 
                kernel_size=11, 
                strides=4, 
                padding='same'),
    pool_size=2,
    name='Coarse1')
Coarse2 = tf.layers.max_pooling2d(
    inputs = tf.layers.conv2d(
                inputs=Coarse1,
                filters=256,
                kernel_size=5,
                padding='same'),
    pool_size=2,
    name='Coarse2')
Coarse3 = tf.layers.conv2d(
            inputs = Coarse2,
            filters=384,
            kernel_size=3,
            padding='same',
            name='Coarse3')
Coarse4 = tf.layers.conv2d(
            inputs = Coarse3,
            filters=384,
            kernel_size=3,
            padding='same',
            name='Coarse4')
Coarse5 = tf.layers.max_pooling2d(
    inputs = tf.layers.conv2d(
                inputs = Coarse4,
                filters=256,
                kernel_size=3,
                padding='same',),
    pool_size=2,
    name='Coarse5')
Coarse6 = tf.layers.dense()
